In [1]:
import pandas as pd
import numpy as np
from math import sin, cos, pi, atan2, asin, sqrt, ceil
import matplotlib.pyplot as plt
from model_funcs import *

## "FULLSCALE" 3

In [35]:
SIFT1_datafile = "Full_FS3_SIFT1.csv"
imu_data_file_and_path = "../../" + SIFT1_datafile

final_grid_number = calc_displacement2(imu_data_file_and_path, 210, [-2, 0], my_thresh=50, my_post_drogue_delay=0.85, my_signal_length=3, my_t_sim_landing=50, ld_launch_angle=2*pi/180, ld_ssm=3.2, ld_dry_base=15.89)

Setting maxIndex to -1
Take-Off Time (s) = 0.0
Landing (s) = 73.71000000000004
End time of signal: 11.559999999999945
End time of signal: 14.559999999999945
IMU WIND SPEEDS, X->1.9757579530014833 m/s and Y->0.29483364599852413 m/s
UPDATED WIND SPEEDS, X->-1.915579462793291 m/s and Y->0.29483364599852413 m/s
LOOP 2
2.915579462793291
Model2 x displacement: 32.466079245801055
Model2 y displacement: 0
AFTER POSSIBLE SIGN FLIP
Model2 x displacement: -32.466079245801055
Model2 y displacement: 0
MODEL 1: TOTAL X AND Y DISPLACEMENTS, u=-1: X->-156.716136 m, Y->25.878550 m
MODEL 2: TOTAL X AND Y DISPLACEMENTS, u=-1: X->-172.00010568638132 m, Y->0 m

1.915579462793291
Model2 x displacement: 31.74901032674729
Model2 y displacement: 0
AFTER POSSIBLE SIGN FLIP
Model2 x displacement: -31.74901032674729
Model2 y displacement: 0
MODEL 1: TOTAL X AND Y DISPLACEMENTS, u=0: X->-109.677156 m, Y->25.878550 m
MODEL 2: TOTAL X AND Y DISPLACEMENTS, u=0: X->-124.24405639066714 m, Y->0 m

0.9155794627932909
Mod

In [37]:
# The actual flight time was ~75 seconds, not 50...
final_grid_number = calc_displacement2(imu_data_file_and_path, 210, [-2, 0], my_thresh=50, my_post_drogue_delay=0.85, my_signal_length=3, my_t_sim_landing=75, ld_launch_angle=2*pi/180, ld_ssm=3.2, ld_dry_base=15.89)

Take-Off Time (s) = 0.0
Landing (s) = 73.71000000000004
End time of signal: 11.559999999999945
End time of signal: 14.559999999999945
IMU WIND SPEEDS, X->1.9757579530014833 m/s and Y->0.29483364599852413 m/s
UPDATED WIND SPEEDS, X->-1.915579462793291 m/s and Y->0.29483364599852413 m/s
LOOP 2
2.915579462793291
Model2 x displacement: 32.466079245801055
Model2 y displacement: 0
AFTER POSSIBLE SIGN FLIP
Model2 x displacement: -32.466079245801055
Model2 y displacement: 0
MODEL 1: TOTAL X AND Y DISPLACEMENTS, u=-1: X->-156.716136 m, Y->25.878550 m
MODEL 2: TOTAL X AND Y DISPLACEMENTS, u=-1: X->-172.00010568638132 m, Y->0 m

1.915579462793291
Model2 x displacement: 31.74901032674729
Model2 y displacement: 0
AFTER POSSIBLE SIGN FLIP
Model2 x displacement: -31.74901032674729
Model2 y displacement: 0
MODEL 1: TOTAL X AND Y DISPLACEMENTS, u=0: X->-109.677156 m, Y->25.878550 m
MODEL 2: TOTAL X AND Y DISPLACEMENTS, u=0: X->-124.24405639066714 m, Y->0 m

0.9155794627932909
Model2 x displacement: 32.

SIFT2

In [38]:
SIFT2_datafile = "Full_FS3_SIFT2.csv"
imu_data_file_and_path = "../../" + SIFT2_datafile

final_grid_number = calc_displacement2(imu_data_file_and_path, 210, [-2, 0], my_thresh=50, my_post_drogue_delay=0.85, my_signal_length=3, my_t_sim_landing=50, ld_launch_angle=2*pi/180, ld_ssm=3.2, ld_dry_base=15.89)

Setting minIndex to -1
Setting maxIndex to -1
Setting lateLandingIndex to -1
Take-Off Time (s) = 0.0


IndexError: index -470976 is out of bounds for axis 0 with size 2885

In [39]:
# The actual flight time was ~75 seconds, not 50...
final_grid_number = calc_displacement2(imu_data_file_and_path, 210, [-2, 0], my_thresh=50, my_post_drogue_delay=0.85, my_signal_length=3, my_t_sim_landing=75, ld_launch_angle=2*pi/180, ld_ssm=3.2, ld_dry_base=15.89)

Setting minIndex to -1
Setting maxIndex to -1
Setting lateLandingIndex to -1
Take-Off Time (s) = 0.0


IndexError: index -470976 is out of bounds for axis 0 with size 2885

## "FULLSCALE" 2

In [ ]:
SIFT2_datafile = "Full_FS2_SIFT2.csv"
imu_data_file_and_path = "../../" + SIFT2_datafile

final_grid_number = calc_displacement2(imu_data_file_and_path, 210, my_thresh=50, my_post_drogue_delay=0.85, my_signal_length=3, my_t_sim_landing=50, ld_launch_angle=2*pi/180, ld_ssm=3.2, ld_dry_base=15.89)

## FULLSCALE 1
> This is for Hoppensville, and we don't have a gridded image for this

In [ ]:
Fullscale1_datafile = "Fullscale1.csv"
imu_data_file_and_path = "../Data/" + Fullscale1_datafile

final_grid_number = calc_displacement2(imu_data_file_and_path, 210, my_thresh=50, my_post_drogue_delay=0.85, my_signal_length=3, my_t_sim_landing=90, ld_launch_angle=2*pi/180, ld_ssm=3.2, ld_dry_base=16.57, ld_m_motor=1.76, ld_t_burn=2.1, ld_T_avg=1740)